# Cleanup

### Delete any existing Docker Hub images

In [ ]:
source ./config.sh

set -e

TOKEN=$(curl -s -H "Content-Type: application/json" -X POST -d '{"username": "'${DOCKERHUB_USERNAME}'", "password": "'${DOCKERHUB_PASSWORD}'"}' https://hub.docker.com/v2/users/login/ | jq -r .token)

REPOSITORY_NAME="account-service"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

REPOSITORY_NAME="confirmation-service"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

REPOSITORY_NAME="payment-service"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

REPOSITORY_NAME="dotnet-core-react-example"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

# Create and Configure Clusters

### Create clusters

In [ ]:
source ./config.sh

tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 3 --instance-type m5.xlarge --group ${GROUP} --name ${CICD_CLUSTER}
tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 3 --instance-type m5.xlarge --group ${GROUP} --name ${ALPHA_CLUSTER}
tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 3 --instance-type m5.xlarge --group ${GROUP} --name ${BRAVO_CLUSTER}

### Monitor creation progress

In [ ]:
tmc cluster list --group bush

### Add clusters to kubeconfig file

In [ ]:
source ./config.sh

KUBECONFIG_CICD=`mktemp` && \
tmc cluster provisionedcluster kubeconfig get ${CICD_CLUSTER} >> ${KUBECONFIG_CICD} && \
KUBECONFIG_ALPHA=`mktemp` && \
tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} >> ${KUBECONFIG_ALPHA} && \
KUBECONFIG_BRAVO=`mktemp` && \
tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} >> ${KUBECONFIG_BRAVO}

if [ $? -eq 0 ]; then
    KUBECONFIG=${KUBECONFIG_CICD}:${KUBECONFIG_ALPHA}:${KUBECONFIG_BRAVO} kubectl config view --merge --flatten > ~/.kube/config

    rm ${KUBECONFIG_CICD}
    rm ${KUBECONFIG_ALPHA}
    rm ${KUBECONFIG_BRAVO}
fi

## Update cluster permissions

In [ ]:
source ./config.sh

kubectl config use-context ${CICD_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated

echo
kubectl config use-context ${ALPHA_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated

echo
kubectl config use-context ${BRAVO_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated

# Install [Argo CD](https://argoproj.github.io/argo-cd/)

In [ ]:
source ./config.sh

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Create Argo CD namespace...'
kubectl create namespace argocd

echo
echo 'Install Argo CD on the CI/CD cluster...'
kubectl -n argocd apply -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml

echo
echo 'Fetch a Load Balanced IP for the Argo CD server...'
kubectl -n argocd patch svc argocd-server -p '{"spec": {"type": "LoadBalancer"}}'

### Wait for load balancer to be provisioned...

In [ ]:
source ./config.sh

kubectl config use-context ${CICD_CLUSTER}

kubectl -n argocd get svc argocd-server
nslookup `kubectl -n argocd get svc/argocd-server -o=jsonpath='{.status.loadBalancer.ingress[0].hostname}'`

### Finish configuring Argo CD

In [ ]:
source ./config.sh

kubectl config use-context ${CICD_CLUSTER}

ARGOCD_USERNAME=admin
ARGOCD_PASSWORD=`kubectl -n argocd get pods -l app.kubernetes.io/name=argocd-server -o name | cut -d'/' -f 2`

echo
echo 'Store the Argo CD host name...'
ARGOCD_HOST=`kubectl -n argocd get svc/argocd-server -o json | jq '.status.loadBalancer.ingress[0].hostname' -j`

./argocd login ${ARGOCD_HOST} --name argocd --username ${ARGOCD_USERNAME} --password ${ARGOCD_PASSWORD} --insecure

argocd cluster add ${ALPHA_CLUSTER}
argocd cluster add ${BRAVO_CLUSTER}

echo
echo "Argo CD Host: $ARGOCD_HOST"
echo "Username: $ARGOCD_USERNAME"
echo "Password: $ARGOCD_PASSWORD"

# Install Kubeapps

### Create service accounts

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}

echo
kubectl create namespace kubeapps
kubectl create -n kubeapps serviceaccount kubeapps-operator
kubectl create clusterrolebinding kubeapps-operator --clusterrole=cluster-admin --serviceaccount=kubeapps:kubeapps-operator

echo
echo 'Token for kubeapps on Alpha cluster:'
kubectl get -n kubeapps secret $(kubectl get -n kubeapps serviceaccount kubeapps-operator -o jsonpath='{.secrets[].name}') -o go-template='{{.data.token | base64decode}}' && echo

echo
kubectl config use-context ${BRAVO_CLUSTER}

echo
kubectl create namespace kubeapps
kubectl create -n kubeapps serviceaccount kubeapps-operator
kubectl create clusterrolebinding kubeapps-operator --clusterrole=cluster-admin --serviceaccount=kubeapps:kubeapps-operator

echo
echo 'Token for kubeapps on Bravo cluster:'
kubectl get -n kubeapps secret $(kubectl get -n kubeapps serviceaccount kubeapps-operator -o jsonpath='{.secrets[].name}') -o go-template='{{.data.token | base64decode}}' && echo

### Configure Argo CD to deploy Kubeapps

In [ ]:
source ./config.sh

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`
BRAVO_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the Kubeapps Helm chart in Argo CD...'
cat ./kubernetes/cicd/argocd/kubeapps.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -
cat ./kubernetes/cicd/argocd/kubeapps.yml | sed "s/CLUSTER_URL/${BRAVO_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/bravo/" | kubectl apply -n argocd -f -

### Get Kubeapps endpoints & login tokens

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}

echo
echo 'Kubeapps on Alpha cluster:'
kubectl get -n kubeapps -o json service kubeapps-alpha | jq '.status.loadBalancer.ingress[0].hostname' -j

echo
echo
echo 'Alpha cluster login token:'
kubectl get -n kubeapps secret $(kubectl get -n kubeapps serviceaccount kubeapps-operator -o jsonpath='{.secrets[].name}') -o go-template='{{.data.token | base64decode}}' && echo

echo
kubectl config use-context ${BRAVO_CLUSTER}

echo
echo 'Kubeapps on Bravo cluster:'
kubectl get -n kubeapps -o json service kubeapps-bravo | jq '.status.loadBalancer.ingress[0].hostname' -j

echo
echo
echo 'Bravo cluster login token:'
kubectl get -n kubeapps secret $(kubectl get -n kubeapps serviceaccount kubeapps-operator -o jsonpath='{.secrets[].name}') -o go-template='{{.data.token | base64decode}}' && echo

### Install RabbitMQ

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}
kubectl create namespace rabbit

cat ./kubernetes/cicd/argocd/rabbit-secret.template.yml | sed "s/RABBITMQ_PASSWORD/$ALPHA_RABBITMQ_PASSWORD/" | kubectl create -n rabbit -f -

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`
BRAVO_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the RabbitMQ Helm chart in Argo CD...'
cat ./kubernetes/cicd/argocd/rabbit.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -


### Install Redis

In [ ]:
source ./config.sh

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`
BRAVO_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${ALPHA_CLUSTER}
kubectl create namespace redis

kubectl config use-context ${CICD_CLUSTER}
cat ./kubernetes/cicd/argocd/redis.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -

### Install Wavefront Proxy

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}
kubectl create namespace wavefront
cat ./kubernetes/cicd/argocd/wavefront-token.template.yml | sed "s/WAVEFRONT_TOKEN/$ALPHA_WAVEFRONT_TOKEN/" | kubectl create -n wavefront -f -

echo
kubectl config use-context ${BRAVO_CLUSTER}
kubectl create namespace wavefront
cat ./kubernetes/cicd/argocd/wavefront-token.template.yml | sed "s/WAVEFRONT_TOKEN/$BRAVO_WAVEFRONT_TOKEN/" | kubectl create -n wavefront -f -

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`
BRAVO_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the Wavefront Proxy Helm chart in Argo CD...'
cat ./kubernetes/cicd/argocd/wavefront-proxy.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -
cat ./kubernetes/cicd/argocd/wavefront-proxy.yml | sed "s/CLUSTER_URL/${BRAVO_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/bravo/" | kubectl apply -n argocd -f -

# Install [kpack](https://tanzu.vmware.com/build-service)

In [ ]:
source ./config.sh

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Install kpack...'
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/kpack-release-0.0.9.yml

echo
echo 'Set the Github and Docker secrets...'
cat ./kubernetes/cicd/kpack/github-creds.template.yml | sed "s/GITHUB_USERNAME/$GITHUB_USERNAME/" | sed "s/GITHUB_PASSWORD/$GITHUB_PASSWORD/" | kubectl create -n kpack -f -
cat ./kubernetes/cicd/kpack/dockerhub-creds.template.yml | sed "s/DOCKERHUB_USERNAME/$DOCKERHUB_USERNAME/" | sed "s/DOCKERHUB_PASSWORD/$DOCKERHUB_PASSWORD/" | kubectl create -n kpack -f -

echo
echo 'Install the default kpack builder...'
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/default-builder.yml

In [ ]:
source ./config.sh

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the application images...'
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/images/account-image.yml
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/images/confirmation-image.yml
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/images/payment-image.yml
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/images/dotnet-image.yml
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/images/spring-music-image.yml

# Configure [Tanzu Service Mesh](https://console.cloud.vmware.com)

### Enable Istio automatic sidecar injection

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}
kubectl label namespace default istio-injection=enabled --overwrite

echo
kubectl config use-context ${BRAVO_CLUSTER}
kubectl label namespace default istio-injection=enabled --overwrite

- Onboard alpha & bravo clusters into Tanzu Service Mesh
- Create the "acme.com" global namespace and connect the default namespaces in the alpha & bravo clusters

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}
kubectl create -f ./kubernetes/istio/ingressgateway.yaml
kubectl create -f ./kubernetes/istio/virtualservice-alpha.yaml

echo
kubectl config use-context ${BRAVO_CLUSTER}
kubectl create -f ./kubernetes/istio/ingressgateway.yaml
kubectl create -f ./kubernetes/istio/virtualservice-bravo.yaml

## Configure GitOps for Alpha cluster

In [ ]:
source ./config.sh

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${ALPHA_CLUSTER}

# copy rabbit secrets to default namespace
kubectl get secret rabbit --namespace=rabbit --export -o yaml | kubectl apply --namespace=default -f -

# copy redis secrets to default namespace
kubectl create secret generic redis-alpha --namespace=default --from-literal="redis-password"=`kubectl get --namespace redis secret redis-alpha -o=jsonpath='{.data.redis-password}' | base64 -D`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the applications in Argo CD...'
cat ./kubernetes/cicd/argocd/account-service.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -
cat ./kubernetes/cicd/argocd/confirmation-service.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -
cat ./kubernetes/cicd/argocd/payment-service.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/"  | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -

## Configure GitOps for Bravo cluster

In [ ]:
source ./config.sh

BRAVO_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the applications in Argo CD...'
cat ./kubernetes/cicd/argocd/dotnet-service.yml | sed "s/CLUSTER_URL/${BRAVO_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/bravo/" | kubectl apply -n argocd -f -

## Retrieve the external hostname for the Istio ingress gateway

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}

echo 'Alpha cluster ingress gateway:'
kubectl get service istio-ingressgateway -n istio-system -o=jsonpath='{.status.loadBalancer.ingress[0].hostname}'

echo
echo
kubectl config use-context ${BRAVO_CLUSTER}

echo 'Bravo cluster ingress gateway:'
kubectl get service istio-ingressgateway -n istio-system -o=jsonpath='{.status.loadBalancer.ingress[0].hostname}'

## Delete clusters

In [ ]:
source ./config.sh

tmc cluster delete ${CICD_CLUSTER}
tmc cluster delete ${ALPHA_CLUSTER}
tmc cluster delete ${BRAVO_CLUSTER}

### To Do
- Add links to running applications
- Rework the readme file
- Use Harbor registry to demonstrate CVEs in public images
- Use [direnv](https://direnv.net/)?